In [1]:
# import libraries
import FinanceDataReader as fdr
import pandas as pd
import numpy as np
import os
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from tqdm import tqdm

C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
#########사용법#########
"""
기본적인 code들은 영서님과 제가 업로드한 code를 이용해서 구현했습니다.
우선, 사용할 모델과 학습 데이터셋을 결정하시고 나서 해당 code를 수정해서 돌리면 됩니다.
해당 code는 ./Frame./test_codes./모델_데이터셋.ipynb 형태로 저장해서 돌리시면 됩니다.

해당 모델은 xgboost로 월~금의 데이터를 기반으로 다음주 월, 화, 수, 목, 금의 종가를 예측합니다.

다만 2년치 데이터를 목표로 하니까 원인 미상의 오류가 발생하여 최근 1년치밖에 학습하지 못했습니다.
우선 결과값이 그닥 좋은 모습을 보이지 않아, 우선 5일간의 data를 기준으로 학습하는 것을 우선적으로 해보겠습니다
"""
########################

'\n기본적인 code들은 영서님과 제가 업로드한 code를 이용해서 구현했습니다.\n우선, 사용할 모델과 학습 데이터셋을 결정하시고 나서 해당 code를 수정해서 돌리면 됩니다.\n해당 code는 ./Frame./test_codes./모델_데이터셋.ipynb 형태로 저장해서 돌리시면 됩니다.\n\n해당 모델은 xgboost로 월~금의 데이터를 기반으로 다음주 월, 화, 수, 목, 금의 종가를 예측합니다.\n\n다만 2년치 데이터를 목표로 하니까 원인 미상의 오류가 발생하여 최근 1년치밖에 학습하지 못했습니다.\n우선 결과값이 그닥 좋은 모습을 보이지 않아, 우선 5일간의 data를 기준으로 학습하는 것을 우선적으로 해보겠습니다\n'

In [3]:
# 기본적으로 사용하는 함수들
# columns rename function
def cols_rename(data_set, target_name):
    for i in data_set.columns:
        if i == 'Date':
            pass
        else:
            data_set.rename(columns={i:target_name+'_'+i}, inplace=True)
    return data_set
# columns rename function which has integer column name
def cols_int_rename(data_set, target_name):
    for i in data_set.columns:
        if i == 'Date':
            pass
        else:
            data_set.rename(columns={i:target_name+'_'+str(i)}, inplace=True)
    return data_set
# column rename function
def col_rename(data_set, name_from, name_to):
    for i in data_set.columns:
        if i != name_from:
            pass
        else:
            data_set.rename(columns={i:name_to}, inplace=True)
    return data_set


In [4]:
#세팅해야 하는 변수들

# 데이터 수집 기간 설정
#start_date = '20210104'
start_date = '20200105'
#end_date = '20201225'
end_date = '20211126'

# 기본적인 data를 불러오는 folder path
path = '../data'
list_name = 'stock_list.csv'
sample_name = 'sample_submission.csv'

stock_list = pd.read_csv(os.path.join(path, list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list

,종목명,종목코드,상장시장
0,삼성전자,005930,KOSPI
1,SK하이닉스,000660,KOSPI
2,NAVER,035420,KOSPI
3,카카오,035720,KOSPI
4,삼성바이오로직스,207940,KOSPI
...,...,...,...
365,맘스터치,220630,KOSDAQ
366,다날,064260,KOSDAQ
367,제이시스메디칼,287410,KOSDAQ
368,크리스에프앤씨,110790,KOSDAQ


In [5]:
######################################
# 학습시킬 data를 선정하는 code 입니다
######################################

# make business_days: 주말 미포함
# data: 전반적으로 학습시킬 모든 data를 담는 변수라고 보시면 될 것 같습니다.
Business_days = pd.DataFrame(pd.date_range(start_date, end_date, freq='B'), columns = ['Date'])
Business_days = pd.DataFrame(Business_days)
Business_days['weekday'] = Business_days.Date.apply(lambda x: x.weekday())
Business_days['weeknum'] = Business_days.Date.apply(lambda x: x.strftime('%Y_%V'))


################################################################################
# 이 사이에다가 추가하고 싶으신 데이터를 넣으시면 됩니다.
# KOSPI : 최대한 건들지 않는게 좋을 것 같습니다.
KOSPI = fdr.DataReader('KS11', start_date, end_date).reset_index()
KOSPI = KOSPI.drop(['Open', 'High', 'Low'], axis=1)
cols_rename(KOSPI, 'kospi')
data = pd.merge(Business_days, KOSPI, how='outer')

# KOSDAQ 추가
KOSDAQ = fdr.DataReader('KQ11', start_date, end_date).reset_index()
KOSDAQ = KOSDAQ.drop(['Open', 'High', 'Low'], axis=1)
cols_rename(KOSDAQ, 'kosdaq')
data = pd.merge(data, KOSDAQ, how='outer')

# 미국증시: 나스닥(NASDAQ) 추가
NAS = fdr.DataReader('NASDAQCOM', start_date, end_date, data_source='fred').reset_index()
col_rename(NAS, 'DATE', 'Date')
col_rename(NAS, 'NASDAQCOM', 'nasdaq_Close')
data = pd.merge(data, NAS, how='outer')

# 환율: 원달러(USD) 추가
USD = fdr.DataReader('USD/KRW', start_date, end_date).reset_index()
USD = USD.drop(['Open', 'High', 'Low'], axis=1)
cols_rename(USD, 'usd')
data = pd.merge(data, USD, how='outer')

# 환율: 원엔(JPY) 추가
JPY = fdr.DataReader('JPY/KRW', start_date, end_date).reset_index()
JPY = JPY.drop(['Open', 'High', 'Low'], axis=1)
cols_rename(JPY, 'jpy')
data = pd.merge(data, JPY, how='outer')

# 환율: 호주달러/스위스프랑 추가
ACF = fdr.DataReader('AUD/CHF', start_date, end_date).reset_index()
ACF = ACF.drop(['Open', 'High', 'Low'], axis=1)
cols_rename(ACF, 'acf')
data = pd.merge(data, ACF, how='outer')

#10년 단위 미 국채
#US_10 = fdr.DataReader('US10YT=X', start_date, end_date).reset_index()
#US_10 = US_10.drop(['Open', 'High', 'Low'], axis=1)
# 일~금 데이터만 있고 토요일 데이터는 없기 때문에 해당 부분 반영
#cols_rename(US_10, 'us_10')
#data = pd.merge(data, US_10, how='outer')

# 변동성모델(CBOE에서 제공하는)
VIX = fdr.DataReader('VIX', start_date, end_date).reset_index()
VIX = VIX.drop(['Open', 'High', 'Low'], axis=1)
cols_rename(VIX, 'vix')
data = pd.merge(data, VIX, how='outer')




start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%Y_%V')
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])
################################################################################



In [6]:
##############
# 휴장일 데이터에 대해서 처리해야한다.
# 휴장일 data는 삭제한다.
#kor_NAN_list = data[data['kospi_Close'].isnull()].index
#data.drop(set(kor_NAN_list.to_list()), inplace = True)
# 휴장일 데이터는 앞의 데이터를 가져온다
#print(data.isnull().sum())
kor_NAN_list = data[data['kospi_Close'].isnull()].index
en_NAN_list = data[data['vix_Close'].isnull()].index


# learn에 사용할 데이터
#train_data = data.drop(['weekday', 'weeknum'], axis=1)

In [7]:
en_NAN_list

Int64Index([10, 30, 69, 100, 175, 259, 270, 290, 365, 390, 435], dtype='int64')

In [8]:
data = data.fillna(method='ffill')
data = data.fillna(method='bfill')
#nan_list = set(kor_NAN_list.to_list() + en_NAN_list.to_list())
#data.drop(nan_list, inplace=True)
data[data['kospi_Close'].isnull()].index

Int64Index([], dtype='int64')

In [9]:
"""
# 아니 만들고있는데 ffill 잘ㅊ 작동하네 대체 뭘 실수했던거지 ㅂㄷㅂㄷ
# 빈 부분을 채우는 함수. 무조건 숫자여야만 한다!
# ffill, bfill이 작동하지 않고, 또한 채우는 값을 선형적이게 변동하게 하는 방법도 가능하기 때문에 해당 함수를 생성했다.
# 채우고 싶은 column에 따라서, 각각의 null value를 찾아서 채워준다.
# dataframe: 비어있는 부분이 있는 dataframe을 pandas의 dataframe으로 넣기
# fill_column: 채우고 싶은 특정 column을 list 형태로 넣는다. 1개더라도 list 형태로 넣는다.
#              만약 None이라면, 모든 column의 빈 공간을 채운다.
# filltype: 3가지가 존재한다, ffill, bfill, 그리고 linear다. ffill, bfill은 동일하고, linear는 산술평균을 linear하게 넣어준다.
def fill_empty_value(dataframe, fill_column = None, fill_type = 'linear'):
    # dataframe 복사
    data_frame = pd.copy(dataframe)
    def fill_empty_column(column_name, empty_list):
        index_l = 0
        index_r = 0
        if fill_type == 'linear':
            while index_l < len(empty_list):
                while index_r < len(empty_list)-1 and empty_list[index_r]+1 == empty_list[index_r+1]:
                    index_r += 1
                # [index_l:index_r+1]을 기준으로 채운다.
                
        elif fill_type == 'bfill':
            while index_l < len(empty_list):
                while index_r < len(empty_list)-1 and empty_list[index_r]+1 == empty_list[index_r+1]:
                    index_r += 1
                # [index_l:index_r+1]을 기준으로 채운다.
                
                # 맨 처음부터 비어있는 경우는 뒤의 데이터로 채워준다.
                if index_l == 0:
                    data_frame[column_name].loc[index_l:index_r+1] = data_frame[column_name].loc[index_r+1]
                # 맨 끝부분을 채울 때는 따로 계산해준다
                elif index_r == len(empty_list)-1:
                    data_frame[column_name].loc[index_l:index_r+1] = data_frame[column_name].loc[index_r+1]
                else:
                    data_frame[column_name].loc[index_l:index_r+1] = data_frame[column_name].loc[index_r+1]
        elif fill_type == 'ffill':
            while index_l < len(empty_list):
                while index_r < len(empty_list)-1 and empty_list[index_r]+1 == empty_list[index_r+1]:
                    index_r += 1
                # [index_l:index_r+1]을 기준으로 채운다.
        else:
            raise Exception('올바른 fill_type가 아닙니다')
    
    index_l = 0
    index_r = 0
    
    # 모든 column들에 대해 빈 내용을 채운다.
    if fill_column == None:
        
    
    
    
    while index_l < len(kor_NAN_list):
        index_r = index_l

        # 연결된 휴일이 있다면 끝까지 연결
        while index_r < len(kor_NAN_list)-1 and kor_NAN_list[index_r]+1==kor_NAN_list[index_r+1]:
            index_r += 1
        #print(f"range is [{index_l}:{index_r}+1]")
        #[index_l:index_r+1]을 기준으로 잡힌다. 

        # 해당 값을 채우는 방법으로, 

        index_r += 1
        index_l = index_r
"""

'\n# 아니 만들고있는데 ffill 잘ㅊ 작동하네 대체 뭘 실수했던거지 ㅂㄷㅂㄷ\n# 빈 부분을 채우는 함수. 무조건 숫자여야만 한다!\n# ffill, bfill이 작동하지 않고, 또한 채우는 값을 선형적이게 변동하게 하는 방법도 가능하기 때문에 해당 함수를 생성했다.\n# 채우고 싶은 column에 따라서, 각각의 null value를 찾아서 채워준다.\n# dataframe: 비어있는 부분이 있는 dataframe을 pandas의 dataframe으로 넣기\n# fill_column: 채우고 싶은 특정 column을 list 형태로 넣는다. 1개더라도 list 형태로 넣는다.\n#              만약 None이라면, 모든 column의 빈 공간을 채운다.\n# filltype: 3가지가 존재한다, ffill, bfill, 그리고 linear다. ffill, bfill은 동일하고, linear는 산술평균을 linear하게 넣어준다.\ndef fill_empty_value(dataframe, fill_column = None, fill_type = \'linear\'):\n    # dataframe 복사\n    data_frame = pd.copy(dataframe)\n    def fill_empty_column(column_name, empty_list):\n        index_l = 0\n        index_r = 0\n        if fill_type == \'linear\':\n            while index_l < len(empty_list):\n                while index_r < len(empty_list)-1 and empty_list[index_r]+1 == empty_list[index_r+1]:\n                    index_r += 1\n                # [index_l:index_r+1]을 기준으로 채운다.\n                \n 

In [10]:
# sample submission
sample_submission = pd.read_csv(os.path.join(path, sample_name))
submission = sample_submission.set_index('Day')
# test day list: 월 화 수 목 금이라고 가정
day_list = ['2021-11-29', '2021-11-30', '2021-12-01', '2021-12-02', '2021-12-03']
#last_index = sample_submission.shape[0]
#sample_submission.drop(range(0, last_index), axis=0, inplace=True)
#sample_submission.loc[day_list[0]]
sample_submission

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,...,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2021-11-02,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2021-11-03,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2021-11-04,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2021-11-05,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2021-11-29,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,2021-11-30,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,2021-12-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,2021-12-02,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,2021-12-03,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
##############################################
# 특정 model에 대해서, 각 종목별로 학습하는 code 입니다
##############################################
import warnings
warnings.filterwarnings('ignore')

sample_name = 'sample_submission.csv'
sample_submission = pd.read_csv(os.path.join(path, sample_name))
submission = sample_submission.set_index('Day')
day_list = ['2021-11-29', '2021-11-30', '2021-12-01', '2021-12-02', '2021-12-03']
#day_list = ['2021-12-06', '2021-12-07', '2021-12-08', '2021-12-09', '2021-12-10']

stock_list.drop(stock_list[stock_list['종목코드']=='031390'].index,inplace=True)
day_first = day_list[0]
ms_data = data.copy()
ms_data.drop(['weeknum'], axis=1, inplace=True)
ms_data.columns

Index(['Date', 'weekday', 'kospi_Close', 'kospi_Volume', 'kospi_Change',
       'kosdaq_Close', 'kosdaq_Volume', 'kosdaq_Change', 'nasdaq_Close',
       'usd_Close', 'usd_Change', 'jpy_Close', 'jpy_Change', 'acf_Close',
       'acf_Change', 'vix_Close', 'vix_Volume', 'vix_Change'],
      dtype='object')

In [12]:
data.columns

Index(['Date', 'weekday', 'weeknum', 'kospi_Close', 'kospi_Volume',
       'kospi_Change', 'kosdaq_Close', 'kosdaq_Volume', 'kosdaq_Change',
       'nasdaq_Close', 'usd_Close', 'usd_Change', 'jpy_Close', 'jpy_Change',
       'acf_Close', 'acf_Change', 'vix_Close', 'vix_Volume', 'vix_Change'],
      dtype='object')

In [13]:
#submission = submission.iloc[5:,:]
true_close = sample_submission.copy().set_index('Day')

code_data = fdr.DataReader('000060', start = '2021-11-01', end = '2021-11-01')['Close'].reset_index()

for code in tqdm(stock_list['종목코드'].values):
    for day in day_list:
        code_data = fdr.DataReader(code, start = day, end = day)['Close'].reset_index()
        true_close.loc[day,code] = code_data['Close'].values[0]

100%|████████████████████████████████████████████████████████████████████████████████| 369/369 [02:18<00:00,  2.66it/s]


In [14]:
##############################################
# 특정 model에 대해서, 각 종목별로 학습하는 code 입니다
##############################################
import warnings
warnings.filterwarnings('ignore')

sample_name = 'sample_submission.csv'
sample_submission = pd.read_csv(os.path.join(path, sample_name))
submission = sample_submission.set_index('Day')
day_list = ['2021-11-29', '2021-11-30', '2021-12-01', '2021-12-02', '2021-12-03']
#day_list = ['2021-12-06', '2021-12-07', '2021-12-08', '2021-12-09', '2021-12-10']
stock_list.drop(stock_list[stock_list['종목코드']=='031390'].index,inplace=True)
day_first = day_list[0]

In [22]:
# 종목별로 사용할 데이터 작성
# train_x[종목코드]를 하면 해당하는 numpy 배열이 저장되어있다.
train_x = dict()
train_y = dict()
test_x = dict()
test_y = dict()
total_x = dict()
total_y = dict()

# 데이터 정리
for code in tqdm(stock_list['종목코드'].values):
    xgb = XGBRegressor()
    
    # col이 월~금, row가 weeknum으로 구분되는 data이다.
    # 해당 data를 y값으로 train할 것이다.
    code_data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()

    y_data = pd.merge(Business_days, code_data, how='outer')
    y_data['weekday'] = y_data.Date.apply(lambda x : x.weekday())
    y_data['weeknum'] = y_data.Date.apply(lambda x : x.strftime('%Y_%V'))
    y_data.Close = y_data.Close.ffill()
    y_data = pd.pivot_table(data = y_data, values='Close', columns='weekday', index='weeknum').to_numpy()
    
    # x에 대해서 작성
    # y처럼 pivot table 작성 후, 1줄 shift하여 저번주의 data와, 저번 주의 추가요소들을 이용해서 다음 주 통째로 예측하는 방식으로 해보자
    # numpy로 저장
    
    code_data = fdr.DataReader(code, start = start_date, end = end_date).reset_index()
    code_data = pd.merge(Business_days, code_data, how='outer')
    
    #이동 평균선 추가: 5, 10, 20, 60일(1주 전)
    #code_data['5_close'] = code_data['Close'].rolling(window=5).mean()   
    #code_data['10_close'] = code_data['Close'].rolling(window=10).mean()   
    #code_data['20_close'] = code_data['Close'].rolling(window=20).mean()
    #code_data['60_close'] = code_data['Close'].rolling(window=60).mean() 
    df = pd.merge(ms_data, code_data, how='left', on='Date')
    df['weekday'] = df.Date.apply(lambda x : x.weekday())
    df['weeknum'] = df.Date.apply(lambda x : x.strftime('%Y_%V'))
    
    #print(pd.isna(df.to_numpy()).sum())
    df = df.ffill()
    #df = df.bfill()
    
    # 만약 KOSPI면, kosdaq 관련 내용 drop
    market = stock_list[stock_list['종목코드'].str.contains(code)]['상장시장'].values[0]
    if market == 'KOSPI':
        df = df.drop(['kosdaq_Close', 'kosdaq_Volume', 'kosdaq_Change'], axis=1)
    else:
        df = df.drop(['kospi_Close', 'kospi_Volume', 'kospi_Change'], axis=1)
        
    # weekday()로 하면, 2021.01.01이 2021_53으로 저장됨, 사실은 2020_53이기 때문에, 해당 부분을 바꿔줘야 한다.
    # 찾아서 직접 바꾸자
    df.columns = df.columns.get_level_values(0)
    df[df['weeknum'] == '2021_53']['weeknum'] = '2020_53'
    # Business day의 시작 날짜보다, 상장한 날짜가 더 느린 경우를 생각하기 위해, close 기준으로 앞 부분 다 빼주기
    print(df[(df['weeknum'] == '2021_53')]['weeknum'])
    df = df[df['Close'].notna()]
    
    #df.to_csv("./test.csv")
    #print(df.columns[2:15].to_list() + df.columns[16:].to_list())
    a = df.columns.to_list()
    a.remove('Date')
    a.remove('weekday')
    a.remove('weeknum')
    
    #print(pd.pivot_table(data=df, values=a, columns='weekday', index='weeknum'))
    break
    x_data = pd.pivot_table(data=df, values=a, columns='weekday', index='weeknum').to_numpy()
    #print(pd.isna(x_data).sum())
    
    #pd.pivot_table(data=df, values=df.columns[2:15].to_list() + df.columns[16:].to_list(), columns='weekday', index='weeknum').to_csv("./pivot.csv")
    #print(pd.isna(x_data).sum())
    
    # 전체를 저장하는 변수에 해당 부분 저장
    #total_y[code] = y_data[1:-1]
    #total_x[code] = x_data[0:-2]
    
    
    #break
    # 이제 학습 후 다음 5일을 예측한다
    #print(x_data[0:-2].shape)
    #print(y_data[1:-1][:,0].shape)
    ind_ = 0
    for day in day_list:
        xgb.fit(x_data[0:-2], y_data[1:-1][:,ind_])
        predict = xgb.predict(x_data[None,-1])
        submission.loc[day, code] = predict
        ind_ += 1        
    # 


  0%|                                                                                          | 0/369 [00:00<?, ?it/s]

259    2021_53
Name: weeknum, dtype: object


In [ ]:
np.mean(np.mean(np.abs(submission[5:] - true_close[5:]) / true_close[5:]))*100

In [ ]:
true_close[5:]

In [ ]:
submission[5:]